In [1]:
from dt_class import DecisionTree

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [2]:
RANDOM_STATE = 14

In [3]:
tree = DecisionTree(max_depth=3, debug=True)

DecisionTree class set params:
{'max_depth': 3, 'min_samples_split': 2, 'criterion': 'gini', 'debug': True}


In [4]:
X, y = make_classification(n_features=2, n_redundant=0, n_samples=1000, random_state=RANDOM_STATE)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

tree.fit(X_train, y_train)

node (depth=1 n_objects=800) division: feature_idx = 1 threshold = 0.11
node (depth=2 n_objects=466) division: feature_idx = 1 threshold = -0.66
create leaf (depth=3 n_objects=300): value = 0 labels_ratio = [0.91 0.09]
create leaf (depth=3 n_objects=166): value = 0 labels_ratio = [0.6 0.4]
node (depth=2 n_objects=334) division: feature_idx = 0 threshold = -2.39
create leaf (depth=3 n_objects=11): value = 0 labels_ratio = [0.64 0.36]
create leaf (depth=3 n_objects=323): value = 1 labels_ratio = [0.06 0.94]


DecisionTree(criterion='gini', debug=True, max_depth=3, min_samples_split=2)